In [367]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv("C:\\Users\\Asus\\Desktop\debt_euro.csv")

In [ ]:
# Annual value matrix

years = np.sort(df['year'].unique())
firms = np.sort(df['companyid'].unique())
debt_types = np.sort(df['descriptiontext'].unique())
value_matrices = []

for year in years:
    value_matrix = pd.DataFrame(np.zeros((len(firms), len(debt_types))), index=firms, columns=debt_types)
    total = []
    
    for firm in firms:
        
        for debt_type in debt_types: 
            
            value = sum(df.loc[(df['companyid'] == firm) & (df['year'] == year) & (df['descriptiontext'] == debt_type)]['dataitemvalue'])
            
            if value > 0:
                value_matrix.loc[firm, debt_type] = value 
            else:
                value_matrix.loc[firm, debt_type] = 0
                
        total_debt = np.sum(value_matrix.loc[firm, :])
        total.append(total_debt)
    
    value_matrix['Total debt'] = pd.Series(total, index=value_matrix.index)
    value_matrices.append(value_matrix)
    
annual_value_matrix = dict(zip(years, value_matrices))

In [ ]:
annual_value_matrix[2001]

In [ ]:
# Annual proportions matrix

prop_matrices = []

for value_matrix in value_matrices:
    prop_matrix = value_matrix
    for firm in firms:
        if value_matrix.loc[firm, 'Total debt'] > 0:
            prop_matrix.loc[firm] = value_matrix.loc[firm] / value_matrix.loc[firm, 'Total debt']
        else:
            prop_matrix.loc[firm] = 0
    prop_matrices.append(prop_matrix)
        
annual_prop_matrix = dict(zip(years, prop_matrices))

In [ ]:
annual_prop_matrix[2001].loc[:,'Capital leases':'Capital leases']

In [ ]:
# Vector of average proportions

avg_proportion_series = []

for year in years:
    proportion = annual_prop_matrix[year]
    proportion = proportion.loc[proportion['Total debt'] > 0]
    avg = pd.Series([proportion.loc[:, debt_type].mean() for debt_type in debt_types], index=debt_types)
    avg_proportion_series.append(avg)
    
avg_proportion = dict(zip(years, avg_proportion_series))

In [ ]:
# Histogram of proportions by year

i = 1
for year in years:
    plt.figure(i, figsize=(40, 10))
    plt.bar(debt_types, avg_proportion[year], fill=False, ecolor='black', hatch='/')
    plt.ylabel('% total debt', fontsize=20)
    plt.title('Average capital structure for year {0}'.format(year), fontsize=20)
    i += 1
    plt.show()